In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pytensor.tensor as pt
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import os
import sys
from torch.utils.data import DataLoader
from patsy import dmatrix

sys.path.append(os.path.abspath('../src'))
from data_tools.data_utils import ReportingDataset, create_data_split
project_dir = project_dir = Path.cwd().parent
trunc_D = 40

In [ ]:


create_data_split(
    start_year=2013,
    end_year=2020,
    D=trunc_D,
    data_folder_path = project_dir / "data",
    input_filename="DENGSP.csv",
    train_prop=0.7,
    val_prop=0.2,
    test_prop=0.1
)
complete_df = pd.read_csv(project_dir / "data" / "model" / "complete_data.csv",
    parse_dates=True,
    date_format="%Y %b",
    index_col=0,
    dtype={"value": float},
)

/Users/jamieharris/Documents/GitHub/Imperial/Dengue-Nowcasting-Thesis/src/data_tools/data_utils.py:21: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(base_folder_path, file)) for file in files]
/Users/jamieharris/Documents/GitHub/Imperial/Dengue-Nowcasting-Thesis/src/data_tools/data_utils.py:21: DtypeWarning: Columns (22,44,45,46,54,56,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(base_folder_path, file)) for file in files]
/Users/jamieharris/Documents/GitHub/Imperial/Dengue-Nowcasting-Thesis/src/data_tools/data_utils.py:21: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(os.path.join(base_folder_path, file)) for file in files]
/Users/jamieharris/Documents/GitHub/Imperial/Dengue-Nowcasting-Thesis/src/data_tools/data_u

In [3]:
y_true = complete_df.sum(axis=1)
y_true.index = pd.to_datetime(y_true.index)
y_true

NameError: name 'complete_df' is not defined

In [6]:
train_df = pd.read_csv(project_dir / "data" / "model" / "training_data.csv",
    parse_dates=True,
    date_format="%Y %b",
    index_col=0,
    dtype={"value": float},
)
val_df = pd.read_csv(project_dir / "data" / "model" / "validation_data.csv",
    parse_dates=True,
    date_format="%Y %b",
    index_col=0,
    dtype={"value": float},
)

In [7]:
def mask_matrix(matrix, D):
        masked_matrix = matrix.copy()
        nrow, ncol = masked_matrix.shape

        for i in range(nrow):
            for j in range(ncol):
                if i + j > D - 1:
                     masked_matrix[i, j] = False
        return  masked_matrix

model_D = 40
M = 0
mask = np.ones(shape=(model_D, model_D), dtype=bool)
mask = mask_matrix(mask, model_D)

reporting_triangle = np.zeros(shape=(len(train_df)-model_D, model_D, model_D))


k = 15
train_max_val = train_df.values.max()
train_max_val
for t in range(model_D, len(train_df)):
    
    # get reporting triangles for each time points
    matrix = train_df.copy().iloc[t-model_D:t, :model_D]
    matrix_bool = np.ones_like(matrix, dtype=bool)
    matrix[~mask] = 0
    reporting_triangle[t-model_D, :, :] = matrix
    reporting_triangle[t-model_D, :, :] = reporting_triangle[t-model_D, :, :] / train_max_val

print(reporting_triangle.shape)
print(reporting_triangle[-1, :, :])


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:

def silu(x):
    return x * pm.math.sigmoid(x)

def sampler_kwargs():
    return dict(
        nuts_sampler="nutpie",
        chains=2,
        draws=300,
        tune=1000,
        target_accept=0.95,
        max_treedepth=15,
        nuts_sampler_kwargs={"backend": "jax", "gradient_backend": "jax"}
    )

In [ ]:

len_train = 200
train_obs = np.array(train_df.copy().iloc[model_D: model_D + len_train])
train_triangle = reporting_triangle[:len_train, :, :]
t = np.arange(0, len(train_triangle))
t = np.arange(len(train_triangle))  # full time index
t_week = t % 7                      # day of the week number



# Global trend spline with 6 basis funcs and 3 degree polynomial
spline_trend = dmatrix(
    "bs(t, df=10, degree=3, include_intercept=False)", {"t": t}, return_type='dataframe'
)
X_trend = np.asarray(spline_trend)

# cc is syclic cubic spline
spline_week = dmatrix(
    "cc(t_week, df=6)", {"t_week": t_week}, return_type='dataframe'
)
X_week = np.asarray(spline_week)


with pm.Model() as model:

    # Trend spline coefficients
    sigma_trend = pm.HalfNormal("sigma_trend", 3)
    beta_trend = pm.Normal("beta_trend", mu=0, sigma=sigma_trend, shape=X_trend.shape[1])
    
    # Cyclic spline coefficients (seasonality)
    sigma_week = pm.HalfNormal("sigma_week", 3)
    beta_week = pm.Normal("beta_week", mu=0, sigma=sigma_week, shape=X_week.shape[1])
    
    log_lam = pm.Deterministic(
        "log_lam", pm.math.dot(X_trend, beta_trend) + pm.math.dot(X_week, beta_week)
    )
    lam = pm.Deterministic("lam", pm.math.exp(pm.math.clip(log_lam, -5, 10)))


    # Neural net proportions
    T, D, F = train_triangle.shape
    n_hidden = 8

    h0 = pm.math.flatten(train_triangle, ndim=2)  # (T, D*F)
    
    W1 = pm.Normal("W1", 0, 0.3, shape=(D*F, n_hidden))
    b1_net = pm.Normal("b1_net", 0, 0.3, shape=(n_hidden,))
    h1 = silu(pm.math.dot(h0, W1) + b1_net)

    W2 = pm.Normal("W2", 0, 0.3, shape=(n_hidden, D))
    b2_net = pm.Normal("b2_net", 0, 0.3, shape=(D,))
    p_raw = pm.math.dot(h1, W2) + b2_net


    p_raw = pm.math.clip(p_raw, -10, 10)
    p = pm.Deterministic("p", pm.math.softmax(p_raw, axis=1))

    mu = pm.Deterministic("mu", lam[:, None] * p)
    mu_clipped = pm.math.clip(mu, 1e-3, 1e4)

    alpha = pm.Exponential("alpha", 5)
    z = pm.NegativeBinomial("z", mu_clipped, alpha, observed=train_obs)

    idata = pm.sample(**sampler_kwargs())


In [ ]:
train_triangle.shape

In [ ]:
from PNN.NowcastPNN import NowcastPNN
import torch

M = 0
PNN_train_triangle = reporting_triangle[:len_train, :, :]
nowcast_pnn = NowcastPNN(past_units=M+model_D, max_delay=model_D, conv_channels=[16, 1], hidden_units=[16, 8], dropout_probs=[0.3, 0.1])
nowcast_pnn.load_state_dict(torch.load(f"../src/outputs/weights/weights-{M+model_D}-{model_D}"))

# Put model in eval mode with dropout active
nowcast_pnn.eval()
nowcast_pnn.drop1.train()
nowcast_pnn.drop2.train()

n_samples = 1000
preds = np.zeros(shape=(len(train_triangle), n_samples))
for i in range(n_samples):
    matrix = torch.tensor(PNN_train_triangle)
    preds[:, i] = nowcast_pnn(matrix).sample().numpy()
    

PNN_preds = np.quantile(preds, 0.5, axis=1)
PNN_lower = np.percentile(preds, 2.5, axis=1)
PNN_upper = np.percentile(preds, 97.5, axis=1)

In [ ]:
lam_post_samples = az.extract(idata, group='posterior', num_samples=1000)['lam']
lam_med = np.percentile(lam_post_samples, 50, axis=1)
lam_lower = np.percentile(lam_post_samples, 2.5, axis=1)
lam_upper = np.percentile(lam_post_samples, 97.5, axis=1)


y_plot = y_true[model_D:model_D + len_train]
y_plot.index = pd.to_datetime(y_plot.index)


plt.plot(y_plot.index, y_plot, label='y')

plt.plot(y_plot.index, lam_med, label='BNN', color='green')
# plt.fill_between(y_plot.index, lam_lower, lam_upper, color='green', alpha=0.2, label='BNN 95% CI')



plt.plot(y_plot.index, PNN_preds, label='PNN', color='orange')
# plt.fill_between(y_plot.index, PNN_lower, PNN_upper, color='orange', alpha=0.2, label='PNN 95% CI')


plt.legend()
plt.tight_layout()
plt.title("Number of Dengue Fever Cases in Sao Paulo, Brazil")
plt.xlabel("Date of First Symptom")
plt.ylabel("Case Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
train_posterior = az.extract(idata, group='posterior', num_samples=1000)

def get_train_post(var):
    return np.percentile(train_posterior[var], 50, axis=-1)





In [ ]:
def get_BNN_nowcast(t, len_eval, progressbar):
    eval_triangle = reporting_triangle[t - len_eval: t, :, :]
    eval_obs = eval_triangle[-1, :, :] * train_max_val


    t = np.arange(0, len(eval_triangle))
    t = np.arange(len(eval_triangle))  # full time index
    t_week = t % 7                      # day of the week number

    mask = np.ones_like(eval_obs, dtype=bool)
    mask[-model_D:] = mask_matrix(mask[-model_D:], model_D)



    # Global trend spline with 6 basis funcs and 3 degree polynomial
    spline_trend = dmatrix(
        "bs(t, df=10, degree=3, include_intercept=False)", {"t": t}, return_type='dataframe'
    )
    X_trend = np.asarray(spline_trend)

    # cc is syclic cubic spline
    spline_week = dmatrix(
        "cc(t_week, df=6)", {"t_week": t_week}, return_type='dataframe'
    )
    X_week = np.asarray(spline_week)


    with pm.Model() as model:

        # Trend spline coefficients
        sigma_trend = pm.HalfNormal("sigma_trend", 3)
        beta_trend = pm.Normal("beta_trend", mu=0, sigma=sigma_trend, shape=X_trend.shape[1])
        
        # Cyclic spline coefficients (seasonality)
        sigma_week = pm.HalfNormal("sigma_week", 3)
        beta_week = pm.Normal("beta_week", mu=0, sigma=sigma_week, shape=X_week.shape[1])
        
        log_lam = pm.Deterministic(
            "log_lam", pm.math.dot(X_trend, beta_trend) + pm.math.dot(X_week, beta_week)
        )
        lam = pm.Deterministic("lam", pm.math.exp(pm.math.clip(log_lam, -5, 10)))


        # Neural net proportions
        T, D, F = eval_triangle.shape
        n_hidden = 8
        net_sd = 0.1

        h0 = pm.math.flatten(eval_triangle, ndim=2)  # (T, D*F)
        
        W1 = pm.Normal("W1", get_train_post("W1"), net_sd, shape=(D*F, n_hidden))
        b1_net = pm.Normal("b1_net", get_train_post("b1_net"), net_sd, shape=(n_hidden,))
        h1 = silu(pm.math.dot(h0, W1) + b1_net)

        W2 = pm.Normal("W2", get_train_post("W2"), net_sd, shape=(n_hidden, D))
        b2_net = pm.Normal("b2_net", get_train_post("b2_net"), net_sd, shape=(D,))
        p_raw = pm.math.dot(h1, W2) + b2_net


        p_raw = pm.math.clip(p_raw, -10, 10)
        p = pm.Deterministic("p", pm.math.softmax(p_raw, axis=1))

        mu = pm.Deterministic("mu", lam[:, None] * p)
        mu_clipped = pm.math.clip(mu, 1e-3, 1e4)

        alpha = pm.Exponential("alpha", 2)
        z = pm.NegativeBinomial("z", mu_clipped[mask], alpha, observed=eval_obs[mask])

        idata = pm.sample(progressbar=progressbar, **sampler_kwargs())
        return idata


In [ ]:
def get_PNN_nowcast(eval_range):
    PNN_train_triangle = reporting_triangle[eval_range, :, :]
    nowcast_pnn = NowcastPNN(past_units=M+model_D, max_delay=model_D, conv_channels=[16, 1], hidden_units=[16, 8], dropout_probs=[0.3, 0.1])
    nowcast_pnn.load_state_dict(torch.load(f"../src/outputs/weights/weights-{M+model_D}-{model_D}"))

    # Put model in eval mode with dropout active
    nowcast_pnn.eval()
    nowcast_pnn.drop1.train()
    nowcast_pnn.drop2.train()

    n_samples = 1000
    preds = np.zeros(shape=(len(PNN_train_triangle), n_samples))
    for i in range(n_samples):
        matrix = torch.tensor(PNN_train_triangle)
        preds[:, i] = nowcast_pnn(matrix).sample().numpy()
        

    PNN_preds = np.quantile(preds, 0.5, axis=1)
    PNN_lower = np.percentile(preds, 2.5, axis=1)
    PNN_upper = np.percentile(preds, 97.5, axis=1)
    return PNN_preds

In [ ]:
from PNN.NowcastPNN import NowcastPNN
import torch

M = 0
start_idx = 40
len_eval = 40
idata = get_BNN_nowcast(start_idx, len_eval, progressbar=True)


PNN_train_triangle = reporting_triangle[start_idx - len_eval:start_idx, :, :]
nowcast_pnn = NowcastPNN(past_units=M+model_D, max_delay=model_D, conv_channels=[16, 1], hidden_units=[16, 8], dropout_probs=[0.3, 0.1])
nowcast_pnn.load_state_dict(torch.load(f"../src/outputs/weights/weights-{M+model_D}-{model_D}"))

# Put model in eval mode with dropout active
nowcast_pnn.eval()
nowcast_pnn.drop1.train()
nowcast_pnn.drop2.train()

n_samples = 1000
preds = np.zeros(shape=(len(train_triangle), n_samples))
for i in range(n_samples):
    matrix = torch.tensor(PNN_train_triangle)
    preds[:, i] = nowcast_pnn(matrix).sample().numpy()
    

PNN_preds = np.quantile(preds, 0.5, axis=1)
PNN_lower = np.percentile(preds, 2.5, axis=1)
PNN_upper = np.percentile(preds, 97.5, axis=1)

lam_post_samples = az.extract(idata, group='posterior', num_samples=1000)['lam']
lam_med = np.percentile(lam_post_samples, 50, axis=1)
lam_lower = np.percentile(lam_post_samples, 2.5, axis=1)
lam_upper = np.percentile(lam_post_samples, 97.5, axis=1)


y_plot = y_true[model_D + start_idx - len_eval:model_D + start_idx]
y_plot.index = pd.to_datetime(y_plot.index)


plt.plot(y_plot.index, y_plot, label='y')

plt.plot(y_plot.index, lam_med, label='BNN', color='green')
# plt.fill_between(y_plot.index, lam_lower, lam_upper, color='green', alpha=0.2, label='BNN 95% CI')



plt.plot(y_plot.index, PNN_preds, label='PNN', color='orange')
# plt.fill_between(y_plot.index, PNN_lower, PNN_upper, color='orange', alpha=0.2, label='PNN 95% CI')


plt.legend()
plt.tight_layout()
plt.title("Number of Dengue Fever Cases in Sao Paulo, Brazil")
plt.xlabel("Date of First Symptom")
plt.ylabel("Case Count")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Same day nowcast

BNN_preds = []


eval_range = list(range(40, 100))
for t in eval_range:
    print(t)
    idata = get_BNN_nowcast(t, len_eval=40, progressbar=False)
    lam_post_samples = az.extract(idata, group='posterior', num_samples=1000)['lam']
    lam_pred = np.percentile(lam_post_samples, 50, axis=1)[-1]
    BNN_preds.append(lam_pred)

PNN_preds = get_PNN_nowcast(eval_range)


y_plot = y_true[np.array(eval_range) + model_D]
y_plot.index = pd.to_datetime(y_plot.index)


plt.plot(y_plot.index, y_plot, label='y')

plt.plot(y_plot.index, BNN_preds, label='BNN', color='green')
# plt.fill_between(y_plot.index, lam_lower, lam_upper, color='green', alpha=0.2, label='BNN 95% CI')



plt.plot(y_plot.index, PNN_preds, label='PNN', color='orange')
# plt.fill_between(y_plot.index, PNN_lower, PNN_upper, color='orange', alpha=0.2, label='PNN 95% CI')


plt.legend()
plt.tight_layout()
plt.title("Number of Dengue Fever Cases in Sao Paulo, Brazil")
plt.xlabel("Date of First Symptom")
plt.ylabel("Case Count")
plt.xticks(rotation=45)
plt.show()